# **INTRODUCTIONS**

---

Used for exceptional processes.

The code is separated into multiple sections:

1. **[Views](#views)**
2. **[Datatino Configurations](#datatino-configurations)**

# **DEPENDENCIES**

---

```json
{
    "depends-on": [
        "src/utils/schemas.ipynb",
        "src/utils/conversions.ipynb",
        "src/utils/protos.ipynb"
    ]
}
```

# **TABLES**

---


In [ ]:
-- COPYRIGHT (C) 2020 DE STAAT DER NEDERLANDEN, MINISTERIE VAN   VOLKSGEZONDHEID, WELZIJN EN SPORT.
-- LICENSED UNDER THE EUROPEAN UNION PUBLIC LICENCE V. 1.2 - SEE HTTPS://GITHUB.COM/MINVWS/NL-CONTACT-TRACING-APP-COORDINATIONFOR MORE INFORMATION.

SET ANSI_NULLS ON
GO

SET QUOTED_IDENTIFIER ON
GO

-- 1) CREATE TABLE(S).....
IF NOT EXISTS (SELECT * FROM SYS.TABLES WHERE object_id = OBJECT_ID('VWSSTAGE.MANUAL_INPUT_REPEATING_SHOTS'))
CREATE TABLE [VWSSTAGE].[MANUAL_INPUT_REPEATING_SHOTS](
	[ID] [INT] IDENTITY (1,1) NOT NULL,
	[CATEGORY] [NVARCHAR](255) NULL,
	[KEY] [NVARCHAR](255) NULL,
	[VALUE] [NVARCHAR](255) NULL,
  	[DATE_LAST_INSERTED] [DATETIME] NOT NULL DEFAULT GETDATE()
);
GO

-- 2) CREATE STAGE INDEX(ES).....
IF NOT EXISTS (SELECT * FROM SYS.INDEXES WHERE [NAME] = 'IX_STAGE_MANUAL_INPUT_REPEATING_SHOTS')
CREATE NONCLUSTERED INDEX [IX_STAGE_MANUAL_INPUT_REPEATING_SHOTS] ON [VWSSTAGE].[MANUAL_INPUT_REPEATING_SHOTS]
(
    [DATE_LAST_INSERTED] ASC
)WITH (STATISTICS_NORECOMPUTE = OFF, DROP_EXISTING = OFF, ONLINE = OFF, OPTIMIZE_FOR_SEQUENTIAL_KEY = OFF) ON [PRIMARY]
GO

# **VIEWS**
---

### REPEATING SHOTS ADMINISTERED

In [ ]:

-- COPYRIGHT (C) 2020 DE STAAT DER NEDERLANDEN, MINISTERIE VAN   VOLKSGEZONDHEID, WELZIJN EN SPORT.
-- LICENSED UNDER THE EUROPEAN UNION PUBLIC LICENCE V. 1.2 - SEE HTTPS://GITHUB.COM/MINVWS/NL-CONTACT-TRACING-APP-COORDINATIONFOR MORE INFORMATION.

CREATE OR ALTER  VIEW [VWSDEST].[V_MANUAL_REPEATING_SHOTS_ADMINISTERED]
AS
	WITH CTE AS (
		SELECT *
		FROM [VWSSTAGE].[MANUAL_INPUT_REPEATING_SHOTS]
    WHERE [DATE_LAST_INSERTED] = (SELECT MAX([DATE_LAST_INSERTED]) FROM [VWSSTAGE].[MANUAL_INPUT_REPEATING_SHOTS])
		AND CATEGORY = 'HERHAALPRIK_ADMINISTERED'
	)
    SELECT
        -- Select columns,
		Top (1)
		(SELECT CAST([VALUE] AS INT) FROM CTE WHERE [KEY] = 'GGD_ADMINISTERED_TOTAL')  AS GGD_ADMINISTERED_TOTAL,
		dbo.[CONVERT_DATETIME_TO_UNIX] (CONVERT([DATETIME], (SELECT [VALUE] FROM CTE WHERE [KEY] = 'DATE_UNIX'), 105)) AS DATE_UNIX,
        dbo.[CONVERT_DATETIME_TO_UNIX] ((SELECT [DATE_LAST_INSERTED] FROM CTE WHERE [KEY] = 'DATE_UNIX')) AS [DATE_OF_INSERTION_UNIX]
    FROM CTE
GO

# **DROP ARTIFACTS**

---


### WORKFLOWS

In [ ]:
-- COPYRIGHT (C) 2020 DE STAAT DER NEDERLANDEN, MINISTERIE VAN   VOLKSGEZONDHEID, WELZIJN EN SPORT.
-- LICENSED UNDER THE EUROPEAN UNION PUBLIC LICENCE V. 1.2 - SEE HTTPS://GITHUB.COM/MINVWS/NL-CONTACT-TRACING-APP-COORDINATIONFOR MORE INFORMATION.

DECLARE @workflow_name NVARCHAR(50) = 'MANUAL INSERT REPEATING SHOTS DATA',
        @proto_name VARCHAR(50) = 'NL';

-- 1) DROP PROTOS CONFIGURATION(S).....
DELETE FROM [DATATINO_PROTO_1].[CONFIGURATIONS]
WHERE [VIEW_ID] IN (
    SELECT [ID] AS VID FROM  [DATATINO_PROTO_1].[VIEWS]
    WHERE TRIM([NAME]) IN (
        'VWSDEST.V_MANUAL_REPEATING_SHOTS_ADMINISTERED'
    ))
    AND [PROTO_ID] IN (
        SELECT [ID] FROM [DATATINO_PROTO_1].[PROTOS] protos 
        WHERE [NAME] = @proto_name
    ) AND [NAME] IN (
        'repeating_shots_administered',
        'repeating_shot_administered'
    );

DELETE FROM [DATATINO_PROTO_1].[VIEWS]
WHERE [ID] IN (
    SELECT 
        [ID] AS VID 
    FROM  [DATATINO_PROTO_1].[VIEWS]
    WHERE TRIM([NAME]) IN (
        'VWSDEST.V_MANUAL_REPEATING_SHOTS_ADMINISTERED'
    )
)

-- 2) DROP PROCESS(ES)......
;WITH CTE 
AS
(
    SELECT
        v_processes.[PROCESS_NAME],
        processes.[ID]
    FROM [DATATINO_ORCHESTRATOR_1].[PROCESSES] processes
    INNER JOIN [DATATINO_ORCHESTRATOR_1].[V_WORKFLOWS] v_workflows ON v_workflows.[ID] = processes.[WORKFLOW_ID]
    INNER JOIN [DATATINO_ORCHESTRATOR_1].[V_PROCESSES] v_processes ON v_processes.[PROCESS_ID] = processes.[ID]
    WHERE v_processes.[PROCESS_NAME] IN (
        'PROCESS_LOAD_MANUAL_INSERT_REPEATING_SHOTS_DATA'
    ) AND v_workflows.[NAME] = @workflow_name
)
DELETE [DATATINO_ORCHESTRATOR_1].[PROCESSES] 
FROM [DATATINO_ORCHESTRATOR_1].[DATAFLOWS] dataflows, [DATATINO_ORCHESTRATOR_1].[PROCESSES] processes
WHERE dataflows.[NAME] IN (SELECT [PROCESS_NAME] FROM CTE) 
    AND processes.[ID] IN (SELECT [ID] FROM CTE);

-- 4) DROP SOURCE(S).....
DELETE FROM [DATATINO_ORCHESTRATOR_1].[SOURCES]
WHERE [NAME] IN (
    'SOURCE_LOAD_MANUAL_INSERT_REPEATING_SHOTS_DATA'
);

-- 5) DROP WORKFLOW(S).....
DELETE [DATATINO_ORCHESTRATOR_1].[WORKFLOWS]
FROM [DATATINO_ORCHESTRATOR_1].[DATAFLOWS] dataflows, [DATATINO_ORCHESTRATOR_1].[WORKFLOWS] workflows
WHERE dataflows.[NAME] = @workflow_name
    AND workflows.[ID] IN (
        SELECT
            workflows.[ID]
        FROM [DATATINO_ORCHESTRATOR_1].[WORKFLOWS] workflows
        INNER JOIN [DATATINO_ORCHESTRATOR_1].[V_WORKFLOWS] v_workflows ON v_workflows.[DATAFLOW_ID] = workflows.[DATAFLOW_ID] 
            AND v_workflows.[ID] = workflows.[ID]
        WHERE v_workflows.[NAME] = @workflow_name
    );

-- 6) DROP DATAFLOW(S).....
DELETE [DATATINO_ORCHESTRATOR_1].[H_DATAFLOWS]
WHERE [NAME] IN (
    'PROCESS_LOAD_MANUAL_INSERT_REPEATING_SHOTS_DATA',
    @workflow_name
);

DELETE  [DATATINO_ORCHESTRATOR_1].[DATAFLOWS]
WHERE [NAME] IN (
    'PROCESS_LOAD_MANUAL_INSERT_REPEATING_SHOTS_DATA',
    @workflow_name
)
GO

# **DATATINO CONFIGURATIONS**
---

### WORKFLOWS

In [ ]:
-- COPYRIGHT (C) 2020 DE STAAT DER NEDERLANDEN, MINISTERIE VAN   VOLKSGEZONDHEID, WELZIJN EN SPORT.
-- LICENSED UNDER THE EUROPEAN UNION PUBLIC LICENCE V. 1.2 - SEE HTTPS://GITHUB.COM/MINVWS/NL-CONTACT-TRACING-APP-COORDINATIONFOR MORE INFORMATION.

DECLARE @workflow_name NVARCHAR(50) = 'MANUAL INSERT REPEATING SHOTS DATA'

-- 1) UPSERT WORKFLOW(S).....
EXECUTE [DATATINO_ORCHESTRATOR_1].[UPSERT_WORKFLOW]
    @id = null, 
    @workflow_name = @workflow_name,
    @description = 'MANUAL INSERT OF REPEATING SHOTS DATA', 
    @schedule = '0 14 * * MON-FRI', -- DAILY AT 01:00 PM
    @active = 1;

-- 2) UPSERT SOURCE(S).....
EXECUTE [DATATINO_ORCHESTRATOR_1].[UPSERT_SOURCE] 
    @id = null,
    @source_name = 'SOURCE_LOAD_MANUAL_INSERT_REPEATING_SHOTS_DATA',
    @description = 'SOURCE MANUAL INSERT OF REPEATING SHOTS DATA',
    @source = 'datafiles/VWS/Handmatige_invoer_Herhaalprik_data.csv',
    @source_columns = 'CATEGORY|KEY|VALUE',
    @target_columns = 'CATEGORY|KEY|VALUE',
    @target_name = 'VWSSTAGE.MANUAL_INPUT_REPEATING_SHOTS',
    @source_type = 'CsvFile',
    @location_type = 'AzureBlob',
    @delimiter_type = 'SemiColon',
    @security_profile= 'RIVM';

-- 3) UPSERT PROCESS(ES).....
DECLARE @process_id INT,
        @process_name NVARCHAR(256),
        @process_description NVARCHAR(256),
        @process_source_name VARCHAR(256);

-- 3.1) UPSERT TABLE PROCESS(ES): STAGING......
SET @process_name = 'PROCESS_LOAD_MANUAL_INSERT_REPEATING_SHOTS_DATA';
SET @process_description = CONCAT('LOAD: ', @workflow_name);
SET @process_source_name = 'SOURCE_LOAD_MANUAL_INSERT_REPEATING_SHOTS_DATA';

SELECT TOP(1)
    @process_id = processes.[ID]
FROM [DATATINO_ORCHESTRATOR_1].[PROCESSES] processes
INNER JOIN [DATATINO_ORCHESTRATOR_1].[V_WORKFLOWS] v_workflows ON v_workflows.[ID] = processes.[WORKFLOW_ID]
INNER JOIN [DATATINO_ORCHESTRATOR_1].[V_PROCESSES] v_processes ON v_processes.[PROCESS_ID] = processes.[ID]
WHERE v_processes.[PROCESS_NAME] = @process_name 
    AND v_workflows.[NAME] = @workflow_name;

EXECUTE [DATATINO_ORCHESTRATOR_1].[UPSERT_PROCESS]
    @id = @process_id,
    @process_name = @process_name,
    @description = @process_description,
    @source_name = @process_source_name,
    @schedule = '* * * * *',
    @workflow_name = @workflow_name,
    @active = 1;  
GO

### PROTOS

In [ ]:
-- COPYRIGHT (C) 2020 DE STAAT DER NEDERLANDEN, MINISTERIE VAN   VOLKSGEZONDHEID, WELZIJN EN SPORT.
-- LICENSED UNDER THE EUROPEAN UNION PUBLIC LICENCE V. 1.2 - SEE HTTPS://GITHUB.COM/MINVWS/NL-CONTACT-TRACING-APP-COORDINATIONFOR MORE INFORMATION.

DECLARE @workflow_name NVARCHAR(50) = 'MANUAL INSERT REPEATING SHOTS DATA';

--VIEW THIRD_SHOT_ADMINISTERED
-- 1) UPSERT PROTO VIEW(S).....
    EXECUTE [DATATINO_PROTO_1].[UPSERT_VIEW] 
    @id = null,
    @view_name ='VWSDEST.V_MANUAL_REPEATING_SHOTS_ADMINISTERED',
    @description = 'VIEW OF V_MANUAL_REPEATING_SHOTS_ADMINISTERED',
    @last_update_name = 'DATE_OF_INSERTION_UNIX',
    @constraint_key_name = null,
    @constraint_value = null,
    @grouped_key_name = null,
    @grouped_last_update_name = null;

-- 2) UPSERT PROTO CONFIGURATION(S).....
EXECUTE [DATATINO_PROTO_1].[UPSERT_CONFIGURATION]
    @id = null,
    @proto_name = 'NL',
    @description = 'VIEW CONFIGURATION FOR VWSDEST.V_MANUAL_REPEATING_SHOTS_ADMINISTERED',
    @view_name = 'VWSDEST.V_MANUAL_REPEATING_SHOTS_ADMINISTERED',
    @item_name = 'repeating_shot_administered',
    @constrained = '0',
    @grouped = '0',
    @columns = '*',
    @mapping = '=LOWER()',
    @layout_type_id = '1',
    @active = '1';